# Multilayer Perceptron (MLP) for Battery Management System (BMS) SOH Estimation

<img src="../../doc/img/MachineLearningNetwork.png"
     alt="Markdown Monster icon"
     style="float: left; margin-right: 10px;" />

# Importing Libraries

# Import Data

# Data Preprocessing

We have to add new columns to the dataset that are the previous values of V, I and T. This will allow our model to detect the trend/gradient of the data.